# Potential Energy Surface Derived Property Predictions with MLIPs

Here we will use a relatively new package called `MatCalc` (same developers as `Pymatgen`) that can predict materials properties that can be derived from the potential energy surface (PES) facilitated by the MLIPs.

## Property Predictions with `MatCalc`

We will use the MACE MLIP here.

In [ ]:
from mace.calculators import mace_mp
from matcalc import EOSCalc
from ase.build import bulk
import numpy as np

# Load small, medium, or large MACE model; autodetect device; use float64 precision
calc = mace_mp(model="small", device="", default_dtype="float64")

structure = bulk("Al", "fcc", a=4.05)

# MatCalc equation of state
eosc = EOSCalc(calc)
eos_props = eosc.calc(structure)
print(eos_props.keys())

In [ ]:
print(eos_props["eos"])  # Full EOS data (volumes, energies)
# convert bulk modulus from GPa to eV/Å^3
bm_gpa = eos_props["bulk_modulus_bm"]
eV_per_A3_per_GPa = 160.2176621
bm_ev_per_A3 = bm_gpa / eV_per_A3_per_GPa
print(f"Bulk modulus: {bm_gpa:.6f} GPa = {bm_ev_per_A3:.6f} eV/Å^3")

from pymatgen.analysis.eos import BirchMurnaghan
bm = BirchMurnaghan(eos_props["eos"]["volumes"], eos_props["eos"]["energies"])
bm.fit()  # Fitted EOS parameters
v0 = bm.v0  # Equilibrium volume from fit
print(f"{v0=:.3f} Å³")
a0 = (bm.v0 * (2 ** 0.5)) ** (1 / 3)  # Lattice parameter from fitted equilibrium volume
print(f"{a0=:.3f} Å")
e0 = bm.e0  # Minimum energy from fit
print(f"{e0=:.3f} eV/atom")  # Primitive cell has 1 atom

Compare this to the DFT ground-truth values from the Materials Project [link](https://next-gen.materialsproject.org/materials/mp-134):

a0 = 2.86 A<br>
V0 = 16.51335 A^3<br>
B_0 = 0.4611 eV/A^3

Let us plot the equation of state (energies vs. volumes).

In [ ]:
import matplotlib.pyplot as plt

eos = eos_props["eos"]
volumes = np.array(eos["volumes"])
energies = np.array(eos["energies"])

plt.figure(figsize=(6, 4))
plt.plot(volumes, energies, "o-", markersize=6, label="EOS data")
plt.axvline(v0, color="k", linestyle="--", label=f"Equilibrium V = {v0:.3f} Å³")
plt.xlabel("Volume")
plt.ylabel("Energy")
plt.title("Energy vs Volume (EOS)")
plt.grid(True)
plt.legend()
plt.show()

There are more properties that can be derived from the PES. This includes phonon-related properties, formation energies, and elasticity tensors (see below), among others.

In [ ]:
from matcalc import ElasticityCalc
# MatCalc elasticity
ec = ElasticityCalc(
    calc,
    relax_structure=False,
    relax_deformed_structures=False,
    use_equilibrium=True,
    fmax=0.05,
)
props = ec.calc(structure)
# Raw tensor as returned by MatCalc (3x3x3x3):
print(props.keys())

In [ ]:
C_raw = np.array(props["elastic_tensor"], dtype=float)
print("Elasticity tensor (Voigt notation):", C_raw, sep="\n")
print(f"Shear modulus G (Voigt–Reuss–Hill average): {props['shear_modulus_vrh']:.3f} eV/Å³")
print(f"Bulk modulus B (Voigt–Reuss–Hill average): {props['bulk_modulus_vrh']:.3f} eV/Å³")
print(f"Young's modulus E: {props['youngs_modulus']:.1f} eV/Å³")

## Exercise 17.1

Pick a different (more complex) material from the Materials Project and compare the bulk modulus values.

Please, fill in the mpid and its chemical composition and the results into [this Google Sheet](https://docs.google.com/spreadsheets/d/1xyZJE2nErCL4HIT6vFfMY9oFyOXoBfHyBRVZDDbI3cA/).

### Specify your API Key

In [ ]:
API_KEY = "your_api_key_here"  # Replace with your Materials Project API key

### Query MP and Compare Bulk Modulus Predictions

In [ ]:
from mp_api.client import MPRester
from mace.calculators import mace_mp
from matcalc import EOSCalc

mpid = "mp-XXXXX"  # Replace with your chosen material ID

with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(material_ids=[mpid], fields=["structure", "bulk_modulus"])
    structure_exercise = docs[0].structure  # type: ignore
    bulk_modulus_mp = docs[0].bulk_modulus  # type: ignore
    # structure_exercise = mpr.get_structure_by_material_id("mp-149")

# Load small, medium, or large MACE model; autodetect device; use float64 precision
calc = mace_mp(model="small", device="", default_dtype="float64")

# MatCalc equation of state
eosc_exercise = EOSCalc(calc)
eos_props_exercise = eosc_exercise.calc(structure_exercise)  # type: ignore
bm_gpa_exercise = eos_props_exercise["bulk_modulus_bm"]
print(f"Bulk modulus (MLIP-calculated): {bm_gpa_exercise:.6f} GPa")
print(f"Bulk modulus (Materials Project): {bulk_modulus_mp} GPa")

## MD Simulation Utilizing MLIPs
Let us now run an MD simulation with near-ab initio accuracy of a molecule adsorbed on a surface.

In [ ]:
from ase.build import fcc111, add_adsorbate, molecule
from ase.constraints import FixAtoms
from ase.visualize import view

# build a Cu(111) slab (3x3 surface cell, 4 layers) with vacuum
slab = fcc111('Cu', size=(4, 4, 5), vacuum=12.0, orthogonal=True)
slab.calc = calc  # attach the MACE calculator
slab_energy = slab.get_potential_energy()

# create a CO molecule
co = molecule('CO')
co_iso = co.copy()
co_iso.center(vacuum=12.0)
co_iso.set_pbc([False, False, False])
co_iso.calc = calc
co_energy = co_iso.get_potential_energy()

# Add molecule to the surface
add_adsorbate(slab, co, height=3.0, position="ontop")

# optionally constrain the bottom layers to mimic a bulk-like slab
z = slab.get_positions()[:, 2]
threshold = z.max() - 3.0  # atoms more than 3.0 Å below the top surface are fixed
mask = z < threshold
slab.set_constraint(FixAtoms(mask=mask))

print(slab)
total_energy_slab_adsorbed = slab.get_potential_energy()
print(f"Adsorption energy = {total_energy_slab_adsorbed - slab_energy - co_energy:.3f} eV")

view(slab, viewer='ngl')

Now the NVT MD simulation with the MACE calculator.

In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nvtberendsen import NVTBerendsen
from ase.io.trajectory import Trajectory
from ase import units
from ase.io import write

# Initialize velocities to correspond to 300 K
MaxwellBoltzmannDistribution(slab, temperature_K=300)

# NVT dynamics for equilibration
dyn = NVTBerendsen(
    slab,
    timestep=2.0 * units.fs,
    temperature_K=300,
    taut=100.0 * units.fs
)

# Run Simulation and log data
def print_status(a=slab):
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    temp = ekin / (1.5 * units.kB)
    print(f"Step {dyn.nsteps:>4}: T = {temp:5.1f} K | Epot = {epot:6.3f} eV/atom | Etot = {(epot+ekin):6.3f} eV/atom")

traj = Trajectory('md.traj', 'w', slab)

dyn.attach(print_status, interval=10)
dyn.attach(traj.write, interval=10)

dyn.run(200)  # Run for 200 steps (≈ 0.4 ps)

In [ ]:
from ase.io import read
from ase.visualize import view

traj_visualize = read("md.traj", index=':')
print(f"Loaded {len(traj_visualize)} frames from 'md.traj'")
view(traj_visualize, viewer="ngl")

## Exercise 17.2

Redo this simulation but for a different transition metal surface and a different molecule that is adsorbed.

Again, please, fill in the transition metal, surface, and molecule and the adsorption energy into [this Google Sheet](https://docs.google.com/spreadsheets/d/1xyZJE2nErCL4HIT6vFfMY9oFyOXoBfHyBRVZDDbI3cA/).